In [ ]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
from bs4 import BeautifulSoup
import re
import requests
import random
import csv
import codecs

# ------无头浏览器获取各页面HTML------
def get_page_html(page):
    try:
        # 设置PhantomJS请求头
        dcap = dict(DesiredCapabilities.PHANTOMJS)
        dcap["phantomjs.page.settings.userAgent"] = (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.6 Safari/537.36")
        driver = webdriver.PhantomJS(desired_capabilities=dcap)
        driver.get("http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-%d.shtml" % page)
        # 随机睡眠 
        time.sleep(random.random()+0.3)
        pageSource = driver.page_source
        bsObj = BeautifulSoup(pageSource,'lxml')
        driver.close()
    except:
        bsObj = None
    return bsObj

# ------获取尾页页码，即最大页数------
def get_max_page():
    bsObj = get_page_html(1)
    max_page_href = bsObj.find('div', {'class': 'p_page'}).find('a', string=re.compile('尾页'))['href']
    max_page_re = re.search(re.compile(r'\d{1,}\.shtml'), max_page_href).group()
    max_page = int(max_page_re[0:len(max_page_re)-6])
    return max_page

# ------获取tbody标题列表------
# 暂时用不上，手动设计标题
def get_tbody_title():
    bsObj = get_page_html(1)
    table_title = []
    title_id_list = bsObj.find('table').find('tr').find_all('th')
    for title in title_id_list:
        table_title.append(title.get_text())
    return table_title

# ------获取投诉简述跳转链接的详细文本------
def get_tsnrANDtshf(url):
    # 初始化投诉内容和投诉回复列表
    tsnrANDtshf_list = []
    # 请求头设置
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.6 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3"}
    try:
        html = session.get(url, headers=headers).content
        bsObj = BeautifulSoup(html,'lxml')
        # bsObj = BeautifulSoup(html,'html.parser')

        # 投诉内容有图片或者视频时，其文本获取方式与投诉回复不一致
        tsnr = bsObj.find('div',{'class':'tsnr'}).find_all('p')[-1].get_text()
        tshf = bsObj.find('div',{'class':'tshf'}).find('p').get_text()

        tsnrANDtshf_list.append(tsnr)
        tsnrANDtshf_list.append(tshf)
    except:
        tsnrANDtshf_list = ['未成功获取', '未成功获取']
    return tsnrANDtshf_list

# ------获取各页面HTML的tbody数据------
def get_tbody_data(bsObj):
    # 初始化各页面投诉记录列表
    record_list = []
    # 各页面表单数据（包含标题）
    data_tbody = bsObj.find('table').find_all('tr')
    # 个页面表单数据（只有记录，不包含标题）
    data_list = data_tbody[1:]
    # 遍历单个页面中的所有投诉数据
    for data in data_list:
        # 初始化单条记录列表
        single_record_list = []
        # 提取单条记录的所有数据
        data_text_list = data.find_all('td')
        # 遍历单条记录的八个常规字段（投诉编号，投诉品牌，投诉车系，投诉车型，投诉简述，典型问题，投诉时间，投诉状态）
        # 并依次存入单条记录列表
        for i in range(len(data_text_list)):
            # 获取每个字段文本值
            field = data_text_list[i].get_text()
            # 依次加入单条记录列表
            single_record_list.append(field)
            # 获取投诉车系网页信息中的各种ID
            if i == 2:
                # 品牌ID
                bid = data_text_list[i]['bid']
                # 车系ID
                sid = data_text_list[i]['sid']
                # 车型ID
                mid = data_text_list[i]['mid']
            # 获取问题简述的网页跳转链接
            if i == 4:
                tsjs_href = data_text_list[i].find('a')['href']
                # 随机睡眠
                time.sleep((random.random()+0.5)*2)
                tsnrANDtshf_list = get_tsnrANDtshf(tsjs_href)
                if tsnrANDtshf_list == ['未成功获取', '未成功获取']:
                    time.sleep(3)
                    tsnrANDtshf_list = get_tsnrANDtshf(tsjs_href)
                tsnr = tsnrANDtshf_list[0]
                tshf = tsnrANDtshf_list[1]
            # 典型问题解析
            if i == 5:
                span_list = data_text_list[i].find_all('span')
                # 典型问题处理字符串
                
                # 初始化质量问题字段
                zlwt_str = ''
                # 初始化服务问题字段
                fwwt_str = ''
                # 区分质量问题与服务问题，并判断其是否有问题说明
                for span in span_list:
                    if span['class'][0] == 'bw':
                        zlwt_str = '%s%s' % (zlwt_str,span.get_text())
                    elif len(span['class']) == 1 and span['class'][0] == 'wt':
                        zlwt_str = '%s:%s;' % (zlwt_str, span.get_text())
                    elif span['class'][0] == 'fwwt':
                        fwwt_str = '%s%s' % (fwwt_str, span.get_text())
                    elif len(span['class']) == 1 and span['class'][0] == 'fw':
                        fwwt_str = '%s:%s;' % (fwwt_str, span.get_text())
        # 单条记录在常规字段之后加入8个附加字段（品牌ID,车系ID，车型ID，问题简述URL，投诉内容，投诉回复，质量问题，服务问题）
        single_record_list.append(bid)
        single_record_list.append(sid)
        single_record_list.append(mid)
        single_record_list.append(tsjs_href)
        single_record_list.append(tsnr)
        single_record_list.append(tshf)
        single_record_list.append(zlwt_str)
        single_record_list.append(fwwt_str)
        #print(single_record_list)
        record_list.append(single_record_list)
    return record_list

def write_csvfile(lists,page_start,page_end):
    csvfile = open('%s_%s_ts.csv' %(page_start,page_end), 'w', newline='', encoding='utf-8')
    writer = csv.writer(csvfile)
    # 标题列表
    title_list = ['投诉编号','投诉品牌','投诉车系','投诉车型','投诉简述','典型问题','投诉时间','投诉状态','品牌ID','车系ID','车型ID','问题简述URL','投诉内容','投诉回复','质量问题','服务问题']
    # 写入标题
    writer.writerow(title_list)
    # 遍历投诉总列表中每个页面的记录列表
    for html_list in lists:
        # 遍历灭个页面记录列表中每条列表记录
        for list in html_list:
            writer.writerow(list)
    csvfile.close()

def utf8_2_gbk(oldfile):
    # 设置新文件名称
    newfile = "gbk_%s" % oldfile
    f = codecs.open(oldfile, 'r', 'utf-8')
    utf_str = f.read()
    f.close()
    out_gbk_str = utf_str.encode('GB18030')
    f = open(newfile,'wb')
    f.write(out_gbk_str)
    f.close()

page_start = 1
page_end = 480
ts_lists = []
try:
    for page in range(page_start,page_end + 1):
        # 设置睡眠
        if page % 5 == 0:
            time.sleep((random.random()+1.5)*10)
        # 获取每页HTML
        bsObj = get_page_html(page)
        # 判断bsObj是否为None
        if bsObj:
            # 获取每页HTML记录列表
            record_list = get_tbody_data(bsObj)
            ts_lists.append(record_list)
            print("第%d页投诉信息已成功采集..." % page)
        else:
            print("------第%d页投诉信息未成功采集！！！------" % page)
finally:
    print("正在写入CSV文件...")
    write_csvfile(ts_lists,page_start,page_end)
    utf8_2_gbk(oldfile='%s_%s_ts.csv' %(page_start,page_end))

print("Game Over!")










In [17]:
'''
未成功获取投诉内容和投诉回复
重新采集
'''

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
from bs4 import BeautifulSoup
import re
import requests
import random
import csv
import codecs


# 请求头设置




# ------获取投诉简述跳转链接的详细文本------
def get_tsnrANDtshf(url):
    # 初始化投诉内容和投诉回复列表
    tsnrANDtshf_list = []
    tsnrANDtshf_list.append(url)
    
    session = requests.Session()
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01"
    }
    
    #代理服务器
    proxyHost = "125.115.90.156"
    proxyPort = "4254"
    proxyMeta = "http://%(host)s:%(port)s" % {
        "host" : proxyHost,
        "port" : proxyPort,
    }
    proxies = {
        "http"  : proxyMeta,
    }


    html = session.get(url, headers=headers,timeout=20)

    if html.status_code == 200:
        bsObj = BeautifulSoup(html.content.decode('GB18030'),'html.parser')
        #print(bsObj)

        # 投诉内容有图片或者视频时，其文本获取方式与投诉回复不一致
        tsnr = bsObj.find('div',{'class':'tsnr'}).find_all('p')[-1].get_text()

        tshf = bsObj.find('div',{'class':'tshf'}).find('p').get_text()

        tsnrANDtshf_list.append(tsnr)
        tsnrANDtshf_list.append(tshf)
    elif html.status_code == 404:
        tsnrANDtshf_list = [url,'页面不存在','页面不存在']
    else:
        tsnrANDtshf_list = [url,'其他错误','其他错误']

    return tsnrANDtshf_list

url_list = [
    'http://www.12365auto.com/zlts/20190722/390690.shtml',
    'http://www.12365auto.com/zlts/20190721/390603.shtml',
    'http://www.12365auto.com/zlts/20190719/390036.shtml',
    'http://www.12365auto.com/zlts/20190717/389667.shtml',
    'http://www.12365auto.com/zlts/20190717/389378.shtml',
    'http://www.12365auto.com/zlts/20190714/388545.shtml',
    'http://www.12365auto.com/zlts/20190713/388261.shtml',
    'http://www.12365auto.com/zlts/20190711/387965.shtml',
    'http://www.12365auto.com/zlts/20190711/387870.shtml',
    'http://www.12365auto.com/zlts/20190711/387750.shtml',
    'http://www.12365auto.com/zlts/20190710/387668.shtml',
    'http://www.12365auto.com/zlts/20190709/387366.shtml',
    'http://www.12365auto.com/zlts/20190708/386884.shtml',
    'http://www.12365auto.com/zlts/20191010/413609.shtml',
    'http://www.12365auto.com/zlts/20191009/413106.shtml',
    'http://www.12365auto.com/zlts/20191008/412880.shtml',
    'http://www.12365auto.com/zlts/20191008/412688.shtml',
    'http://www.12365auto.com/zlts/20191007/412570.shtml',
    'http://www.12365auto.com/zlts/20191007/412416.shtml',
    'http://www.12365auto.com/zlts/20191006/412269.shtml',
    'http://www.12365auto.com/zlts/20191003/411709.shtml',
    'http://www.12365auto.com/zlts/20191003/411701.shtml',
    'http://www.12365auto.com/zlts/20191003/411655.shtml',
    'http://www.12365auto.com/zlts/20191002/411335.shtml',
    'http://www.12365auto.com/zlts/20190929/410732.shtml',
    'http://www.12365auto.com/zlts/20190929/410669.shtml',
    'http://www.12365auto.com/zlts/20190929/410583.shtml',
    'http://www.12365auto.com/zlts/20190927/410007.shtml',
    'http://www.12365auto.com/zlts/20190926/409953.shtml',
    'http://www.12365auto.com/zlts/20190926/409835.shtml',
    'http://www.12365auto.com/zlts/20190925/409480.shtml',
    'http://www.12365auto.com/zlts/20190925/409359.shtml',
    'http://www.12365auto.com/zlts/20190925/409275.shtml',
    'http://www.12365auto.com/zlts/20190924/408937.shtml',
    'http://www.12365auto.com/zlts/20190923/408729.shtml',
    'http://www.12365auto.com/zlts/20190923/408693.shtml',
    'http://www.12365auto.com/zlts/20190923/408628.shtml',
    'http://www.12365auto.com/zlts/20190923/408611.shtml',
    'http://www.12365auto.com/zlts/20190922/408412.shtml',
    'http://www.12365auto.com/zlts/20190920/407989.shtml',
    'http://www.12365auto.com/zlts/20190920/407903.shtml',
    'http://www.12365auto.com/zlts/20190920/407846.shtml',
    'http://www.12365auto.com/zlts/20190918/407343.shtml',
    'http://www.12365auto.com/zlts/20190918/407288.shtml',
    'http://www.12365auto.com/zlts/20190918/407184.shtml',
    'http://www.12365auto.com/zlts/20190916/406722.shtml',
    'http://www.12365auto.com/zlts/20190916/406713.shtml',
    'http://www.12365auto.com/zlts/20190916/406563.shtml',
    'http://www.12365auto.com/zlts/20190913/405995.shtml',
    'http://www.12365auto.com/zlts/20190912/405772.shtml',
    'http://www.12365auto.com/zlts/20190910/405212.shtml',
    'http://www.12365auto.com/zlts/20190910/405180.shtml',
    'http://www.12365auto.com/zlts/20190909/404766.shtml',
    'http://www.12365auto.com/zlts/20190907/404523.shtml',
    'http://www.12365auto.com/zlts/20190907/404476.shtml',
    'http://www.12365auto.com/zlts/20190907/404335.shtml',
    'http://www.12365auto.com/zlts/20190905/403967.shtml',
    'http://www.12365auto.com/zlts/20190905/403962.shtml',
    'http://www.12365auto.com/zlts/20190830/402128.shtml',
    'http://www.12365auto.com/zlts/20190829/402051.shtml',
    'http://www.12365auto.com/zlts/20190829/402045.shtml',
    'http://www.12365auto.com/zlts/20190829/401998.shtml',
    'http://www.12365auto.com/zlts/20190828/401738.shtml',
    'http://www.12365auto.com/zlts/20190826/400863.shtml',
    'http://www.12365auto.com/zlts/20190825/400763.shtml',
    'http://www.12365auto.com/zlts/20190825/400642.shtml',
    'http://www.12365auto.com/zlts/20190824/400478.shtml',
    'http://www.12365auto.com/zlts/20190822/399911.shtml',
    'http://www.12365auto.com/zlts/20190819/399006.shtml',
    'http://www.12365auto.com/zlts/20190819/398863.shtml',
    'http://www.12365auto.com/zlts/20190816/398247.shtml',
    'http://www.12365auto.com/zlts/20190815/398081.shtml',
    'http://www.12365auto.com/zlts/20190814/397671.shtml',
    'http://www.12365auto.com/zlts/20190812/397031.shtml',
    'http://www.12365auto.com/zlts/20190811/396695.shtml',
    'http://www.12365auto.com/zlts/20190809/396247.shtml',
    'http://www.12365auto.com/zlts/20190808/395966.shtml',
    'http://www.12365auto.com/zlts/20190807/395406.shtml',
    'http://www.12365auto.com/zlts/20190806/395240.shtml',
    'http://www.12365auto.com/zlts/20190805/394965.shtml',
    'http://www.12365auto.com/zlts/20190803/394443.shtml',
    'http://www.12365auto.com/zlts/20190803/394325.shtml',
    'http://www.12365auto.com/zlts/20190803/394312.shtml',
    'http://www.12365auto.com/zlts/20190803/394300.shtml',
    'http://www.12365auto.com/zlts/20190803/394260.shtml',
    'http://www.12365auto.com/zlts/20190802/394198.shtml',
    'http://www.12365auto.com/zlts/20190802/394196.shtml',
    'http://www.12365auto.com/zlts/20190802/394167.shtml',
    'http://www.12365auto.com/zlts/20190801/393629.shtml',
    'http://www.12365auto.com/zlts/20190731/393416.shtml',
    'http://www.12365auto.com/zlts/20190730/393284.shtml',
    'http://www.12365auto.com/zlts/20190730/393167.shtml',
    'http://www.12365auto.com/zlts/20190728/392547.shtml',
    'http://www.12365auto.com/zlts/20190727/392348.shtml',
    'http://www.12365auto.com/zlts/20190726/392031.shtml',
    'http://www.12365auto.com/zlts/20190725/391747.shtml',
    'http://www.12365auto.com/zlts/20190722/390690.shtml',
    'http://www.12365auto.com/zlts/20190721/390603.shtml'
]


result = []
count = 0
for url in url_list:
    count += 1
    aaa = get_tsnrANDtshf(url)
    result.append(aaa)
    print('第%s条记录采集完成：%s'%(count,aaa))
    time.sleep(20)
    
print(result)

'''

########################################################

tsnrANDtshf_list = []
tsnrANDtshf_list.append(url)


html = session.get(url, headers=headers).content
html = session.get(url, headers=headers).content.decode('GB18030')

bsObj = BeautifulSoup(html,'html.parser')
#print(bsObj)

# 投诉内容有图片或者视频时，其文本获取方式与投诉回复不一致
tsnr = bsObj.find('div',{'class':'tsnr'}).find_all('p')
print(tsnr)

#tshf = bsObj.find('div',{'class':'tshf'}).find('p').get_text()

tsnrANDtshf_list.append(tsnr)
#tsnrANDtshf_list.append(tshf)
  
print(tsnrANDtshf_list)
'''










第1条记录采集完成：['http://www.12365auto.com/zlts/20190722/390690.shtml', '本人18年1月份提的T90自动智尚版，后来经过一段时间使用，发现车子在发动机较高转速时开着空调制冷,风力在三档以下时，开内循环就会不定时发出股塑胶烧焦气味,车内人闻到这个气味非常的不适，这气味出现无规律,一般几分钟到十多分钟出现一次,特别跑高速时更加严重,以上这些情況如开外循环则会减轻很多或很以闻到异味，但一开内循环就很严重的烧焦气味。用车到现在也有一年半多了，期间去过几次4s店找人处理，但4s店的师傅几次都说闻不到那烧焦味，说我鼻子过于敏感或者心理作用。但这个问题确实存在，而且还越来越严重。本人认为车子空调这个塑胶烧焦的异味，已经严重影响到了本人及车上乘坐人员心身健康。', '车质网已将您的投诉转给生产企业以及政府有关主管部门，我们将会对此投诉继续跟踪，请您持续关注！']
第2条记录采集完成：['http://www.12365auto.com/zlts/20190721/390603.shtml', '三月份买的车到现在，只要停七八天，就出现跑电现象，出现无法发动，这样已经出现五六次了，卖家给我换了电瓶，抜了jps装置，检查了好几次，没有效果，依然出现这种事情，这是产品的质量问题，我要求换车，如果无法协调，我要用法律手段解决。', '广汽乘用车厂家回复：感谢您选择广汽传祺汽车，您的信息我们已经收到，将尽快协调当地销售店与您核实处理，您同时也可拨打广汽传祺客服专线：400-813-6666，我们将竭诚为您服务！']
第3条记录采集完成：['http://www.12365auto.com/zlts/20190719/390036.shtml', '国六新车，仅行驶640公里左右，就出现发动机故障灯亮！论坛和车主群都有类似故障。我暂时还沒去4s店检测。部份车主去4s店消除故障码，暂时灭灯，但开几天又亮回来！论坛说是国六碳罐进水导致，是否是设计问题，要求厂家提供彻底解决的方案！', '广汽丰田厂家回复：我们已收到您的投诉，将尽快对应处理。非常感谢您对广汽丰田的关注和支持，衷心祝您生活愉快，工作顺利！']
第4条记录采集完成：['http://www.12365auto.com/zlts/20190717/389667.shtml', '

第27条记录采集完成：['http://www.12365auto.com/zlts/20190929/410583.shtml', '特别夏天，太阳一嗮，味道更重，坐在车里呼吸辣嗓子，到4s店检查居然说没味道，迟迟没得到解决。', '上汽斯柯达厂家回复：多次致电客户手机均无人应答，故无法联系客户处理。如仍有未尽事宜，可拨打上汽大众客服热线进行反馈，我司会尽快为您协调处理，感谢您的支持！']
第28条记录采集完成：['http://www.12365auto.com/zlts/20190927/410007.shtml', '页面不存在', '页面不存在']
第29条记录采集完成：['http://www.12365auto.com/zlts/20190926/409953.shtml', '您好！我买这辆车时，4S店没有明确告诉我们是库存车，出厂日期到我买车时有8个月了，我于2019年9月20经别人告诉我这车是属于库存车，然后我在网上查阅属实，当日与4S店联系后，受他们邀请于2019年9月25日去他们店协商。上图片两个人和视频是销售经理与我们协商达成补偿是：免费三年六次保养和去4S店每次路上费用，和现时出厂后过了期的机油换掉。可是到了双方签收协议时，4S店老板否认此车不是库存车，不让他手下经理给我办理补偿协议。他说叫我去投诉维权。上图片微信聊天记录是我2019年9月25日晚上回来后和4S店销售总监的聊天记录。她现在又说交车时我看了车，但是作为消费者是不知情，应该告知消费者库存车详细的情况，并双方签交车确认书，这些他们都没有做到属于欺诈消费行为。他们所谓客户现场确认，作为消费者是不懂库存车，只是叫我们验收油漆有无划伤，而且他们明知道是库存车，作为他们销售员工是专业的人士，应该明确告知消费者这是行业规定的库存车，且还要告知库存车存在那些让客户不满意的地方，应该客户在和你们协商同意认可情况下并有双方书面签收确认书，这样才会让消费者沒有异议他们交车现场根本沒有提到库存车的含义是什么，这属于消费欺诈！', '东风乘用车厂家回复：客户反馈的问题厂家已知晓并高度重视，已安排相关4S店和客户进行对接，正在积极的沟通处理中，厂家也将持续关注后续处理结果。感谢客户的支持与谅解。东风风神服务热线：400-880-6600']
第30条记录采集完成：['http://www.12365a

第49条记录采集完成：['http://www.12365auto.com/zlts/20190913/405995.shtml', '自2018年9月购车使用就出现了变速箱偶尔异响，时至2019年9月变速箱异响越发频繁，期间也向一汽奥迪厂家反馈，送4S店检测未检测到异常！近期发现异响主要是变速箱3挡升4挡出现“呯”一声响，但偶发不持续，可能一天才出现一声异响，也可能几天出现一声异响！烦死了，送去检测查不出来，也没办法解决问题！还有驾驶位真皮座椅起皱严重，真心不知道一汽质检干啥去了，这也能出厂售卖！烦！烦！', '车质网已将您的投诉转给生产企业以及政府有关主管部门，我们将会对此投诉继续跟踪，请您持续关注！']
第50条记录采集完成：['http://www.12365auto.com/zlts/20190912/405772.shtml', '2017年2月份买的车，在行驶二万多公里时有“哒哒哒”异响声，属于发动机质量问题。去4S店多次维修，4S店推诿不处理。2019年8月行驶五万多公里时声音更大，去4S店要求处理，4S店换气门挺杆，异响没有解决。现在告诉我需要拆解发动机进一步检查。发动机是车辆心脏，拆解对车辆保价及质量存在较大影响。现在要求更换发动机总成，并延长发动机质保。', '广汽乘用车厂家回复：感谢您选择广汽传祺汽车，您的信息我们已经收到，将尽快协调当地销售店与您核实处理，您同时也可拨打广汽传祺客服专线：400-813-6666，我们将竭诚为您服务！']
第51条记录采集完成：['http://www.12365auto.com/zlts/20190910/405212.shtml', '四个轮胎内侧已磨损报废了，需换四个轮胎，原因是四轮定位数据错误导致，在保质期内质量问题，需换拉扞减震等许多部件，经销商已承认换，但是向厂家申请长时间没有结果。', '众泰汽车厂家回复：您好，您反馈的车辆问题我司已获悉，会联系相关部门反馈核实']
第52条记录采集完成：['http://www.12365auto.com/zlts/20190910/405180.shtml', '发现仪表出现安全气囊紧急维修开回4S店说要更换气车软件。但一直推软件还没有开发岀来。要等三个月。', '车质网已将您的投诉转给生产企业以及政府有关主管部门，我们将会对此投诉继续跟踪，请您持续关注！'

第73条记录采集完成：['http://www.12365auto.com/zlts/20190814/397671.shtml', '本人于2019年6月10在宜宾购买了东风日产奇骏2.0智联舒适版suv一台，6月10号提车当天犹豫天气炎热，一直关窗开空调行驶。第二天用车开窗了发现车辆在行驶中给油引擎盖左右两侧下方，发出嗞嗞嗞异响，松油门则消失，尤其以爬坡异响更为严重，发动机转速一但超过1400转，嗞嗞嗞异响一直存在，原地空挡给油则无此异响，稍后两天就联系了当地4s店到售后检查，售后技主管，售后技师先后在不同路段路试了车，也很明确的听到了嗞嗞嗞异响，然后给我的答复就是反馈厂家就把我打发走了，该到了新车2500公里做首保检查了，我又提出该异响问题，售后技师说等到二保后换机油后看能不能改善，接下来到了5000公里了，二保机油也换了，接下来再次路试，到目前为止嗞嗞嗞异响依然存在，售后技师干脆直接说这个问题无法处理，这车就这样，这里的奇骏2.0的几乎都这样，就是通病了，意思通病就不用管了，我质问售后技师，比如本田1.5t的机油增多是通病了，按照你们的意思就不用召回处理了，这是什么逻辑，售后这下不说话了，给我的答复就是向厂家反应，直到今天8月14号也没有回复我反应的情况，一直就说会向厂家反应，自从发现这个异响开始，奇骏论坛里和好多车友讨论了这个问题，有这个异响的车主真不少，而且几乎都是新车，之所以来投诉，只是希望借助车质网这个平台，能够得到厂家出面解决，不要再继续任由厂商继续坑害消费者，不要再继续纵容他们的不作为。', '东风日产厂家回复：客户反馈的问题厂家已知晓并高度重视，已安排相关4S店和客户进行对接，正在积极的沟通处理中，厂家也将持续关注后续处理结果。感谢客户的支持与谅解，如有疑问请主动致电东风日产热线电话400/8008308899。']
第74条记录采集完成：['http://www.12365auto.com/zlts/20190812/397031.shtml', '10号开车去买菜，买完菜后钥匙插入点火开关后不能启动，致使车辆停了三个多小时，向4s店打电话联系后说是点火开关坏了，朋友跟我说ctv点火开关容易坏我还不相信，严重怀疑本田配套厂的质量问题，还有副驾自从换气嚢发生器后异响严重！原车只有主驾这边响。', '广汽本田厂家回复：您好！您反映的问题已经收到，

第96条记录采集完成：['http://www.12365auto.com/zlts/20190725/391747.shtml', '每次在收油后，再次加油，就在踩油的一瞬间有哒的一声，持续给油没有这个声音，4S店说双离合的车都是这样的，可是试驾店內的试驾车没有这个问题，4S又说是新车磨合期。反应给长城，回复是正常的工作噪音。', '长城汽车厂家回复：经服务商核实车辆属于正常工作噪音。']
第97条记录采集完成：['http://www.12365auto.com/zlts/20190722/390690.shtml', '本人18年1月份提的T90自动智尚版，后来经过一段时间使用，发现车子在发动机较高转速时开着空调制冷,风力在三档以下时，开内循环就会不定时发出股塑胶烧焦气味,车内人闻到这个气味非常的不适，这气味出现无规律,一般几分钟到十多分钟出现一次,特别跑高速时更加严重,以上这些情況如开外循环则会减轻很多或很以闻到异味，但一开内循环就很严重的烧焦气味。用车到现在也有一年半多了，期间去过几次4s店找人处理，但4s店的师傅几次都说闻不到那烧焦味，说我鼻子过于敏感或者心理作用。但这个问题确实存在，而且还越来越严重。本人认为车子空调这个塑胶烧焦的异味，已经严重影响到了本人及车上乘坐人员心身健康。', '车质网已将您的投诉转给生产企业以及政府有关主管部门，我们将会对此投诉继续跟踪，请您持续关注！']
第98条记录采集完成：['http://www.12365auto.com/zlts/20190721/390603.shtml', '三月份买的车到现在，只要停七八天，就出现跑电现象，出现无法发动，这样已经出现五六次了，卖家给我换了电瓶，抜了jps装置，检查了好几次，没有效果，依然出现这种事情，这是产品的质量问题，我要求换车，如果无法协调，我要用法律手段解决。', '广汽乘用车厂家回复：感谢您选择广汽传祺汽车，您的信息我们已经收到，将尽快协调当地销售店与您核实处理，您同时也可拨打广汽传祺客服专线：400-813-6666，我们将竭诚为您服务！']
[['http://www.12365auto.com/zlts/20190722/390690.shtml', '本人18年1月份提的T90自动智尚版，后来经过一段时间使用，发现车子在发动机较高转速时开着空调制冷,风力在三档以

"\n\n########################################################\n\ntsnrANDtshf_list = []\ntsnrANDtshf_list.append(url)\n\n\nhtml = session.get(url, headers=headers).content\nhtml = session.get(url, headers=headers).content.decode('GB18030')\n\nbsObj = BeautifulSoup(html,'html.parser')\n#print(bsObj)\n\n# 投诉内容有图片或者视频时，其文本获取方式与投诉回复不一致\ntsnr = bsObj.find('div',{'class':'tsnr'}).find_all('p')\nprint(tsnr)\n\n#tshf = bsObj.find('div',{'class':'tshf'}).find('p').get_text()\n\ntsnrANDtshf_list.append(tsnr)\n#tsnrANDtshf_list.append(tshf)\n  \nprint(tsnrANDtshf_list)\n"

In [19]:
for i in result:
    print(i)




['http://www.12365auto.com/zlts/20190722/390690.shtml', '本人18年1月份提的T90自动智尚版，后来经过一段时间使用，发现车子在发动机较高转速时开着空调制冷,风力在三档以下时，开内循环就会不定时发出股塑胶烧焦气味,车内人闻到这个气味非常的不适，这气味出现无规律,一般几分钟到十多分钟出现一次,特别跑高速时更加严重,以上这些情況如开外循环则会减轻很多或很以闻到异味，但一开内循环就很严重的烧焦气味。用车到现在也有一年半多了，期间去过几次4s店找人处理，但4s店的师傅几次都说闻不到那烧焦味，说我鼻子过于敏感或者心理作用。但这个问题确实存在，而且还越来越严重。本人认为车子空调这个塑胶烧焦的异味，已经严重影响到了本人及车上乘坐人员心身健康。', '车质网已将您的投诉转给生产企业以及政府有关主管部门，我们将会对此投诉继续跟踪，请您持续关注！']
['http://www.12365auto.com/zlts/20190721/390603.shtml', '三月份买的车到现在，只要停七八天，就出现跑电现象，出现无法发动，这样已经出现五六次了，卖家给我换了电瓶，抜了jps装置，检查了好几次，没有效果，依然出现这种事情，这是产品的质量问题，我要求换车，如果无法协调，我要用法律手段解决。', '广汽乘用车厂家回复：感谢您选择广汽传祺汽车，您的信息我们已经收到，将尽快协调当地销售店与您核实处理，您同时也可拨打广汽传祺客服专线：400-813-6666，我们将竭诚为您服务！']
['http://www.12365auto.com/zlts/20190719/390036.shtml', '国六新车，仅行驶640公里左右，就出现发动机故障灯亮！论坛和车主群都有类似故障。我暂时还沒去4s店检测。部份车主去4s店消除故障码，暂时灭灯，但开几天又亮回来！论坛说是国六碳罐进水导致，是否是设计问题，要求厂家提供彻底解决的方案！', '广汽丰田厂家回复：我们已收到您的投诉，将尽快对应处理。非常感谢您对广汽丰田的关注和支持，衷心祝您生活愉快，工作顺利！']
['http://www.12365auto.com/zlts/20190717/389667.shtml', '本人在广州市番禺区东风风神车友店5月1日购买2019款E70500超亨版，当时沒